In [ ]:
# !pip install avalanche-lib==0.2.0

In [ ]:
import avalanche as avl
import torch
from avalanche.evaluation import metrics as metrics

In [ ]:
model = avl.models.SimpleMLP(
    input_size = 784, 
    hidden_size = 256,
    hidden_layers = 2,
    drop_rate = 0.5,
    num_classes = 10
)

# model = avl.models.as_multitask(model, 'classifier') # convert the SimpleMLP model into a multi-task model

In [ ]:
benchmark = avl.benchmarks.SplitMNIST(
    n_experiences=5,
    return_task_id=False, # set to True for multi-task experiments
    seed=0
)

In [ ]:
interactive_logger = avl.logging.InteractiveLogger()

evaluation_plugin = avl.training.plugins.EvaluationPlugin(
    metrics.accuracy_metrics(experience=True),
    metrics.forgetting_metrics(experience=True),
    loggers=[interactive_logger])

In [ ]:
cl_strategy = avl.training.Naive(
    model = model, 
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001), 
    criterion = torch.nn.CrossEntropyLoss(),
    train_mb_size = 128, 
    train_epochs = 1,
    evaluator = evaluation_plugin)

# cl_strategy = avl.training.Cumulative(
#     model = model, 
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.001), 
#     criterion = torch.nn.CrossEntropyLoss(),
#     train_mb_size = 128, 
#     train_epochs = 1,
#     evaluator = evaluation_plugin)

In [ ]:
results = []
for experience in benchmark.train_stream:
    print("Current Experience: ", experience.current_experience)
    cl_strategy.train(experience)
    results += [cl_strategy.eval(benchmark.test_stream)]

In [ ]:
results[0]

In [ ]:
results[1]

In [ ]:
results[2]

In [ ]:
results[3]

In [ ]:
results[4]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

acc0 = []
acc1 = []
acc2 = []
acc3 = []
acc4 = []

for0 = []
for1 = []
for2 = []
for3 = []

for a in results:
    for b in a:
        c = b.split("/")
        if (c[0] == "Top1_Acc_Exp") and (c[-1]=="Exp000"):
            acc0 += [a[b]]
        elif (c[0] == "Top1_Acc_Exp") and (c[-1]=="Exp001"):
            acc1 += [a[b]]
        elif (c[0] == "Top1_Acc_Exp") and (c[-1]=="Exp002"):
            acc2 += [a[b]]
        elif (c[0] == "Top1_Acc_Exp") and (c[-1]=="Exp003"):
            acc3 += [a[b]]
        elif (c[0] == "Top1_Acc_Exp") and (c[-1]=="Exp004"):
            acc4 += [a[b]]
        elif (c[0] == "ExperienceForgetting") and (c[-1]=="Exp000"):
            for0 += [a[b]]
        elif (c[0] == "ExperienceForgetting") and (c[-1]=="Exp001"):
            for1 += [a[b]]
        elif (c[0] == "ExperienceForgetting") and (c[-1]=="Exp002"):
            for2 += [a[b]]
        elif (c[0] == "ExperienceForgetting") and (c[-1]=="Exp003"):
            for3 += [a[b]]

acc0 = np.array(acc0)
acc1 = np.array(acc1)
acc2 = np.array(acc2)
acc3 = np.array(acc3)
acc4 = np.array(acc4)
for0 = np.array(for0)
for1 = np.array(for1)
for2 = np.array(for2)
for3 = np.array(for3)

In [ ]:
while len(for0) < 5:
    for0 = np.concatenate((np.zeros(1), for0))

while len(for1) < 5:
    for1 = np.concatenate((np.zeros(1), for1))

while len(for2) < 5:
    for2 = np.concatenate((np.zeros(1), for2))
    
while len(for3) < 5:
    for3 = np.concatenate((np.zeros(1), for3))

In [ ]:
experience = np.array(range(5))

In [ ]:
[plt.plot(experience, acc0, label='exp0'),
 plt.plot(experience, acc1, label='exp1'),
 plt.plot(experience, acc2, label='exp2'),
 plt.plot(experience, acc3, label='exp3'),
 plt.plot(experience, acc4, label='exp4'),
 plt.xlabel("training experience"), 
 plt.xticks([0,1,2,3,4]),
 plt.ylabel("experience testing accuracy"),
 plt.figlegend(),
 plt.title("Testing Accuracy")
]

In [ ]:
[plt.plot(experience, for0, label='exp0'),
 plt.plot(experience, for1, label='exp1'),
 plt.plot(experience, for2, label='exp2'),
 plt.plot(experience, for3, label='exp3'),
 plt.xlabel("training experience"), 
 plt.xticks([0,1,2,3,4]),
 plt.ylabel("experience forgetting"),
 plt.figlegend(),
 plt.title("Forgetting score")
]